# Packages 

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *

import joblib
import os
import numpy as np
import scipy
import pandas as pd
from gensim.models import Word2Vec
from tqdm import tqdm
from gensim.similarities.annoy import AnnoyIndexer


from annoy import AnnoyIndex
import polars as pl
import implicit
from src.eval import model_eval

import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from lightgbm import early_stopping
from utils import *


/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


# Config 

In [2]:
candidate_file_name

'{task}_{data_type}_{model_version}_{model_for_eval}_top{topn}.parquet'

In [46]:
debug = False

eval_cg = False

reprocess_train_data = False
train_parts = 20

load_train_eval_data = False
if debug:
    SAMPLE_NUM = 1000
else:
    SAMPLE_NUM = None


candidate_path = '../data/candidates/'
model_dir = '../model_training'
ranker_train_data_dir = '../data/rank_train_data_v2'
# train_data_dir = '.'
# test_data_dir = '.'
task = 'task1'
w2v_model_version = 'w2v_v3'
nic_model_version = 'nic'
nfi_model_version = 'next_few_items_v1'
covisit_model_version = 'co_visit'

# rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
w2v_topn=200
nic_topn=100
nfi_topn=100
covisit_topn = 300
# PREDS_PER_SESSION = 100

# num_tree = 100
# # target locales: locales needed for task1
target_locals = ["DE", 'JP', 'UK']

# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model_version)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")



train_dir = os.path.join(ranker_train_data_dir, 'train')

In [47]:
! mkdir {ranker_train_data_dir}

mkdir: cannot create directory ‘../data/rank_train_data_v2’: File exists


In [48]:
! mkdir {train_dir}

mkdir: cannot create directory ‘../data/rank_train_data_v2/train’: File exists


# Function 

In [41]:
def candidates_validation(target_pl):
    # rec_summary  = (target_pl
    #         .lazy()
    #         .select(
    #             pl.col('next_item_prediction').arr.lengths().alias('rec_num')
    #         ).collect().describe())
    # print(rec_summary)
    max_sample_per_NIP = (
        target_pl.groupby(['session_id', 'next_item_prediction'])
            .agg(
                pl.count().alias('unique_count')
                # , pl.col('w2v_weight').max().alias('max_w2v_weight')
                # , pl.col('w2v_weight').min().alias('min_w2v_weight')
            )
            .select(
                pl.col('unique_count').max().alias('max_unique_cnt')
                # , pl.col('max_w2v_weight').min().alias('min_max_w2v')
                # , pl.col('min_w2v_weight').max().alias('max_min_w2v')
            )
    ).to_pandas()['max_unique_cnt'].values[0]
    # print(stat_info)
    # print(max_sample_per_NIP)
    assert max_sample_per_NIP == 1, f"there should be unique prediction for every session_id"

# Original data 

In [7]:

# eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

# # df_sess.head(3).collect()
# test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'), n_rows=SAMPLE_NUM).with_columns(pl.col('prev_items').apply(str2list))


# Re-process data for memmory 

In [8]:
def re_process_data(task=task, 
                       w2v_model_version=w2v_model_version, 
                       nic_model_version=nic_model_version,
                       nfi_model_version=nfi_model_version,
                       covisit_model_version=covisit_model_version,
                    model_for_eval=model_for_eval,
                      w2v_topn=w2v_topn
                      , nic_topn=nic_topn
                       , nfi_topn=nfi_topn
                       , covisit_topn=covisit_topn
                    , train_parts=train_parts
                      ):
    parts = train_parts

    data_type = 'train'
    w2v_file = os.path.join(candidate_path, 
                           candidate_file_name.format(
                    task=task
                , data_type=data_type
                , model_version=w2v_model_version
                , model_for_eval=model_for_eval
                , topn=w2v_topn
                           ))
    nic_file = os.path.join(candidate_path,
                candidate_file_name.format(
                    task=task
                    , data_type=data_type
                    , model_version=nic_model_version
                    , model_for_eval=model_for_eval
                    , topn=nic_topn
                           ))
    nfi_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=nfi_model_version
                , model_for_eval=model_for_eval
                , topn=nfi_topn
                       ))
    covisit_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=covisit_model_version
                , model_for_eval=model_for_eval
                , topn=covisit_topn
                       ))
    # w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    # nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    # nfi_pl = pl.scan_parquet(nfi_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    # covisit_pl = pl.scan_parquet(covisit_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
    dir_lst = []
    pl_lst = []
    for file_name in [w2v_file, nic_file, nfi_file, covisit_file]:
        data_pl = pl.scan_parquet(file_name, n_rows=SAMPLE_NUM).collect().sort('session_id')
        dir_name = file_name.replace('.parquet', '')
        print(dir_name)
        dir_lst.append(dir_name)
        pl_lst.append(data_pl)
        if not os.path.isdir(dir_name):
            os.mkdir(dir_name)
    assert pl_lst[0].shape[0] == pl_lst[1].shape[0] == pl_lst[2].shape[0] == pl_lst[3].shape[0]
    total_len = pl_lst[0].shape[0]

    rows_per_part = int(total_len/parts)

    for dir_path, data_pl in zip(dir_lst, pl_lst):
        print(dir_path)
        for idx in tqdm(range(parts)):
            begin = idx*rows_per_part
            if idx != parts-1:
                end = (idx+1)*rows_per_part
            else:
                end = total_len
            part_pl = data_pl[begin:end :]
            file_name = os.path.join(dir_path, f"part_{idx+1}.parquet")
            print(f'{begin} - {end} -> {file_name}')
            part_pl.write_parquet(file_name)

In [9]:
reprocess_train_data

False

In [10]:
if reprocess_train_data:
    re_process_data()

In [11]:
# ! ls ../data/candidates/task1_train_w2v_v3_True_top100

In [12]:
# df = pl.scan_parquet(os.path.join('../data/candidates/task1_train_co_visit_True_top300/', 'part_20.parquet'))

In [13]:
# df.head().collect()

# Get candiadtes

In [49]:
# data_type = 'eval'
# w2v_file = os.path.join(candidate_path, 
#                            candidate_file_name.format(
#                     task=task
#                 , data_type=data_type
#                 , model_version=w2v_model_version
#                 , model_for_eval=model_for_eval
#                 , topn=w2v_topn
#                            ))
# w2v_file
# w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)

In [50]:



def get_all_candidates(data_type, task=task, 
                       w2v_model_version=w2v_model_version, 
                       nic_model_version=nic_model_version,
                       nfi_model_version=nfi_model_version,
                       covisit_model_version=covisit_model_version,
                    model_for_eval=model_for_eval,
                      w2v_topn=w2v_topn
                      , nic_topn=nic_topn
                       , nfi_topn=nfi_topn
                       , covisit_topn=covisit_topn
                       , part=None
                      ):
    w2v_file = os.path.join(candidate_path, 
                           candidate_file_name.format(
                    task=task
                , data_type=data_type
                , model_version=w2v_model_version
                , model_for_eval=model_for_eval
                , topn=w2v_topn
                           ))
    nic_file = os.path.join(candidate_path,
                candidate_file_name.format(
                    task=task
                    , data_type=data_type
                    , model_version=nic_model_version
                    , model_for_eval=model_for_eval
                    , topn=nic_topn
                           ))
    nfi_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=nfi_model_version
                , model_for_eval=model_for_eval
                , topn=nfi_topn
                       ))
    covisit_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=covisit_model_version
                , model_for_eval=model_for_eval
                , topn=covisit_topn
                       ))
    print("Candidiate files: ")
    print('+'*20)
    print()
    print(w2v_file)
    print(nic_file)
    print(nfi_file)
    print(covisit_file)
    if data_type == 'train':
        assert part is not None
        w2v_file = os.path.join(w2v_file.replace('.parquet', ''), f"part_{part}.parquet")
        nic_file = os.path.join(nic_file.replace('.parquet', ''), f"part_{part}.parquet")
        nfi_file = os.path.join(nfi_file.replace('.parquet', ''), f"part_{part}.parquet")
        covisit_file = os.path.join(covisit_file.replace('.parquet', ''), f"part_{part}.parquet")

    if data_type == 'test':
        original_file_name = f"sessions_{data_type}_{task}.parquet"
    if data_type == 'test4task3':
        original_file_name = f"sessions_test_task3.parquet"
    if data_type == 'eval':
        original_file_name = f"sessions_{data_type}.parquet"
    original_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, original_file_name), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
    w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    nfi_pl = pl.scan_parquet(nfi_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    covisit_pl = pl.scan_parquet(covisit_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))


#     def get_vector(prev_items):
#         prev_items_vec = np.array([w2vec.wv[ele] for ele in prev_items]).mean(axis=0)
#         return prev_items_vec
    
#     original_pl = (
#             original_pl.with_columns(
#                 pl.col('prev_items').apply(lambda x: get_vector(x)).alias('all_previous_items_vec')
#             )
#         )
    
    # get w2v weight
    w2v_pl = w2v_pl.with_columns(pl.col('next_item_prediction').arr.lengths().alias('rec_num')).with_columns(
        pl.col('rec_num').apply(lambda x: list(range(x, 0, -1))).alias('w2v_weight')
        
    )
    # print(w2v_pl.head().collect())
    get_w2v_weight = pl.element().rank()*0
    nic_pl = nic_pl.with_columns(
        pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                                # parallel=True
                                               ).alias('w2v_weight').cast(pl.List(pl.Int64))
    )
    nfi_pl = nfi_pl.with_columns(
        pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                               ).alias('w2v_weight').cast(pl.List(pl.Int64))
    )
    covisit_pl = covisit_pl.with_columns(
        pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                               ).alias('w2v_weight').cast(pl.List(pl.Int64))
    )
    cols = ['session_id', 'next_item_prediction', 
            'w2v_weight'
           ]
    # combined_pl = (
    #     w2v_pl.select(cols)
    #         .join(nic_pl.select(cols), how='left', on='session_id', suffix='_nic')
    #         .join(nfi_pl.select(cols), how='left', on='session_id', suffix='_nfi')
    # )

    # combined_pl = combined_pl.with_columns(
    #     pl.concat_list([pl.col('next_item_prediction'), pl.col('next_item_prediction_nic')]).alias('next_item_prediction'), 
    #     pl.concat_list([pl.col('w2v_weight'), pl.col('w2v_weight_nic')]).alias('w2v_weight'), 
    # ).select(
    #     pl.all().exclude(['next_item_prediction_nic', 'w2v_weight_nic'])
    # )
    explode_cols = ['next_item_prediction', 
                    'w2v_weight'
                   ]
    combined_pl = (
            pl.concat([w2v_pl.select(cols).explode(explode_cols)
                       , nic_pl.select(cols).explode(explode_cols)
                       , nfi_pl.select(cols).explode(explode_cols)
                       , covisit_pl.select(cols).explode(explode_cols)
                      ], how='vertical')
                .groupby(['session_id', 'next_item_prediction'])
                .agg(
                    pl.col('w2v_weight').max().alias('w2v_weight')
                )
                .join(original_pl.select(['session_id', 'prev_items']), how='left', on='session_id')
                .filter(pl.col('prev_items').arr.contains(pl.col('next_item_prediction'))==False)
                .with_columns(
                    pl.col('w2v_weight').max().over('session_id').alias('max_count')
                    , pl.col('w2v_weight').min().over('session_id').alias('min_count')
                )
                .with_columns(
                    pl.when(pl.col('max_count')==pl.col('min_count')).then(1).otherwise((pl.col('w2v_weight')-pl.col('min_count'))/(pl.col('max_count')-pl.col('min_count'))).alias('w2v_weight')
                )
                .select(
                        pl.exclude(['prev_items', 'max_count', 'min_count'])

                )
                .groupby('session_id')
                .agg(
                    pl.all()
                )
    )
    combined_pl = (
        combined_pl.join(original_pl, how='left', on='session_id')
            .with_columns(
                pl.col('prev_items').arr.lengths().alias('prev_length')
                , pl.col('prev_items').arr.get(-1).alias('last_prev_item')
            ).select(
                pl.all().exclude('prev_items')
            )
    )
    return combined_pl


In [21]:
    # w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))


In [17]:
eval_cg_pl = get_all_candidates(data_type='eval')


Candidiate files: 
++++++++++++++++++++

../data/candidates/task1_eval_w2v_v3_True_top200.parquet
../data/candidates/task1_eval_nic_True_top100.parquet
../data/candidates/task1_eval_next_few_items_v1_True_top100.parquet
../data/candidates/task1_eval_co_visit_True_top300.parquet


In [18]:
! ls ../data/candidates/ | grep test

task1_test4task3_co_visit_True_top300.parquet
task1_test4task3_next_few_items_v1_True_top100.parquet
task1_test4task3_next_item_counter_v2_True_top100.parquet
task1_test_co_visit_True_top300.parquet
task1_test_co_visit_True_top300_phase1.parquet
task1_test_next_few_items_v1_True_top100.parquet
task1_test_next_few_items_v1_True_top100_phase1.parquet
task1_test_nic_True_top100.parquet
task1_test_nic_True_top100_phase1.parquet
task1_test_w2v_v3_True_top100_phase1.parquet
task1_test_w2v_v3_True_top200.parquet
task2_test4task3_co_visit_task2_True_top300.parquet
task2_test4task3_nfi_task2_True_top100.parquet
task2_test4task3_nic_task2_True_top100.parquet
task2_test4task3_w2v_task2_True_top200.parquet
task2_test_co_visit_task2_True_top300.parquet
task2_test_nfi_task2_True_top100.parquet
task2_test_nic_True_top100.parquet
task2_test_nic_task2_True_top100.parquet
task2_test_w2v_task2_True_top200.parquet


In [17]:
test_cg_pl = get_all_candidates(data_type='test')

Candidiate files: 
++++++++++++++++++++

../data/candidates/task1_test_w2v_v3_True_top200.parquet
../data/candidates/task1_test_nic_True_top100.parquet
../data/candidates/task1_test_next_few_items_v1_True_top100.parquet
../data/candidates/task1_test_co_visit_True_top300.parquet


In [48]:
# eval_cg_pl.head().collect()

In [49]:
# eval_cg_pl.select(
#     pl.col('next_item_prediction').arr.lengths().alias('len1')
#     , pl.col('w2v_weight').arr.lengths().alias('len2')
# ).collect().describe()

In [50]:
# eval_cg_pl.select(
#     pl.col('next_item_prediction').arr.lengths().alias('len1')
#     , pl.col('w2v_weight').arr.lengths().alias('len2')
# ).collect().describe()

In [51]:
# ).collect().describe()

In [52]:
# eval_cg_pl.head().collect()

In [53]:
# eval_cg_pl.head().collect()

# Eval candidate generation 

In [54]:
if eval_cg:
    eval_topn = 3000

    col = f"recall@{eval_topn}"
    eval_final = (
            eval_cg_pl
            .lazy()
            .with_columns(
                pl.col('next_item_prediction').cast(pl.List(pl.Utf8))
            )
            ).with_columns(
                pl.col('next_item_prediction').arr.head(eval_topn).arr.contains(pl.col('next_item')).mean().alias(col)

            )
    final_res = eval_final.select(
            pl.count().alias('total_sessions')
            , pl.col(col).mean()

        ).collect()
    describe = (eval_cg_pl
            .lazy()
            .select(
                pl.col('next_item_prediction').arr.lengths().alias('rec_num')
            ).collect().describe())
    print(describe)
    print(final_res)
    del final_res

# Feature Process 

## Load NIC, W2V Model 

In [22]:
nic_model = (
    pl.scan_parquet('../model_training/next_item_counter_v2/nic_model.parquet')
        .explode(['next_item_prediction', 'next_item_weight'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
        )
            )
print(nic_model.schema)

nfi_model = (
    pl.scan_parquet('../model_training/next_few_items_v1/nic_True_for_eval.parquet')
        .explode(['next_item_prediction', 'next_item_weight'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
            # , pl.col()
        )
            )
print(nfi_model.schema)



# w2v_model_file = '../model_training/v2/w2v.model'
w2vec = Word2Vec.load(w2v_model_file)
annoy_index = AnnoyIndexer()
annoy_index.load(annoy_index_file)

{'next_item_prediction': Utf8, 'next_item_weight': Float64, 'last_prev_item': Utf8}
{'next_item_prediction': Utf8, 'next_item_weight': Float32, 'last_prev_item': Utf8}


In [23]:
co_visit_model = (
    pl.scan_parquet(f'../model_training/{covisit_model_version}/{covisit_model_version}_True_for_eval.parquet')
        .explode(['next_item_prediction', 'next_item_weight'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
            # , pl.col()
        )
            )
print(co_visit_model.schema)

{'next_item_prediction': Utf8, 'next_item_weight': Float32, 'last_prev_item': Utf8}


In [24]:
# co_visit_model.head().collect()

## get features 

In [25]:
# target_df = train_candidates
# data_type = 'train'

def get_w2v_simi(x, col='last_prev_item'):
    try:
        if isinstance(x[col], str):
            target = [x[col]]
        else:
            target = x[col]
        simi = w2vec.wv.n_similarity([x['next_item_prediction']],
                                                  target
                                                 )
    except Exception as e:
        print(e)
        # print(x[col])
        simi = 0
    return simi

def get_feature(target_df, data_type, nic_model=nic_model, nfi_model=nfi_model, 
                co_visit_model=co_visit_model,
                negative_sampling=None, w2v_similarity=False):
    if data_type == 'train':
        target_df = target_df.filter(pl.col('next_item_prediction').arr.contains(pl.col('next_item')))
    target_df = target_df.explode(['next_item_prediction', 'w2v_weight'])

    if data_type not in ['test', 'test4task3']:
        target_df = (
            target_df
                .with_columns(
                    pl.when(pl.col('next_item')==pl.col('next_item_prediction')).then(1).otherwise(0).alias('target')
                )
        )

    if negative_sampling is not None:
        cols = ['session_id', 'next_item', 'next_item_prediction', 'target']
        sample_pl = target_df.select(cols)
        pos = sample_pl.filter(pl.col('target')==1)
        neg = (sample_pl
                   .filter(pl.col('target')==0)
              )
        neg_original_len = neg.select('session_id').collect().shape[0]
        print(f'Original negative num: {neg_original_len}')
        # neg = (
        #     neg.with_columns(
        #                 pl.Series(name='random', values=np.random.uniform(size=neg_original_len))
        #            )
        #            .filter(pl.col('random')<=negative_sampling)
        #            .select(pl.all().exclude('random'))
        # )
        
        neg = neg.groupby('session_id').agg(
            pl.all().sample(frac=negative_sampling)
            )
        neg = neg.explode(neg.columns[1:])
        # # print(neg.shape)
        # # print(neg.head().collect())
        sample_pl = pl.concat([pos, neg], how='vertical')
        target_df = sample_pl.join(target_df, how='left', on=cols)#.collect()
        del sample_pl
    target_df = (
            target_df.lazy().join(nic_model, how='left', on=['last_prev_item', 'next_item_prediction'])
                .join(nfi_model, how='left', on=['last_prev_item', 'next_item_prediction'], suffix='_nfi')
                .join(co_visit_model, how='left', on=['last_prev_item', 'next_item_prediction'], suffix='_co_visit')
                .with_columns(
                    pl.when(pl.col('next_item_weight').is_null()).then(0).otherwise(pl.col('next_item_weight')).alias('next_item_weight')
                    , pl.when(pl.col('next_item_weight_nfi').is_null()).then(0).otherwise(pl.col('next_item_weight_nfi')).alias('next_item_weight_nfi')
                    , pl.when(pl.col('next_item_weight_co_visit').is_null()).then(0).otherwise(pl.col('next_item_weight_co_visit')).alias('next_item_weight_co_visit')
                    , pl.struct(["next_item_prediction", "last_prev_item"]).apply(
                        lambda x: get_w2v_simi(x, col='last_prev_item')).alias('last_item_similarity').cast(pl.Float32)
                    # , pl.struct(["next_item_prediction", "prev_items"]).apply(
                    #     lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
                    , pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
                        .then(2)
                        .otherwise(3).alias('locale')
                ).sort('session_id')
                .select(
                    pl.all().exclude('last_prev_item')
                )

    )
    # if w2v_similarity:
    #     target_df = target_df.with_columns(
    #         pl.struct(["next_item_prediction", "prev_items"]).apply(
    #                     lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
    #     )
    return target_df
# target_df.head(3).collect()

In [26]:
# %%time 
# sample_pl = get_feature(target_df=train_cg_pl, data_type='train', #negative_sampling=0.1
#                        ).collect()

In [27]:
# sampled_train_candidates.head().collect()

In [28]:
# part_index = 0
# print()
# file_name = os.path.join(train_dir, f"part_{part_index+1}.parquet")
# print(file_name)
# train_cg_pl = get_all_candidates(data_type='train', part=part_index+1)

In [29]:
# help(pl.all().sample)

In [30]:
# %%time
# sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', negative_sampling=0.1)
# # debug_pl = sampled_train_candidates.head().collect()

In [31]:
# stat_info = (
#         sampled_train_candidates
#             .groupby('session_id')
#             .agg(
#                 pl.count().alias('rec_cnt')
#                 , pl.col('target').sum().alias('target_num')
#             )
#             .collect()
#             .describe()
#     )
# print(stat_info)
# (
#         train_cg_pl.select(
#                 pl.col('next_item_prediction').arr.lengths().alias('rec_num')
#             )
#             .collect()
#             .describe(
#                 # percentiles=[0.1*i for i in range(10)]
#             )
#     )

In [32]:
# stat_info = (
#         sampled_train_candidates
#             .groupby('session_id')
#             .agg(
#                 pl.count().alias('rec_cnt')
#                 , pl.col('target').sum().alias('target_num')
#             )
#             .collect()
#             .describe()
#     )
# print(stat_info)
# (
#         train_cg_pl.select(
#                 pl.col('next_item_prediction').arr.lengths().alias('rec_num')
#             )
#             .collect()
#             .describe(
#                 # percentiles=[0.1*i for i in range(10)]
#             )
#     )

In [29]:
# debug_pl.head()

In [30]:
# %%time
# sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train',
#                                        negative_sampling=0.1,
#                                       w2v_similarity=True)
# debug_pl = sampled_train_candidates.head().collect()

In [31]:
# debug_pl

In [32]:
# prev_items = ["B09W9FND7K", "B09JSPLN1M"]
# next_item_prediction = "B09M7GY217"	
# w2vec.wv.n_similarity(prev_items, [next_item_prediction])
# w2vec.wv.n_similarity(prev_items, prev_items)
# prev_items_vec = np.array([w2vec.wv[ele] for ele in prev_items]).mean(axis=0)
# next_item_vec = w2vec.wv[next_item_prediction]
# 1 - scipy.spatial.distance.cosine(prev_items_vec, next_item_vec)

# Save Data 

## Save eval data 

In [29]:
%load_ext memory_profiler

In [37]:
eval_candidates = get_feature(target_df=eval_cg_pl, data_type='eval')

In [38]:
%%time
%memit
eval_candidates = eval_candidates.collect()

print('Eval data')
print()
print(eval_candidates.shape)
print(f"Memory: {eval_candidates.estimated_size(unit='gb')}")
print(eval_candidates['target'].mean())
print('Validating')
candidates_validation(target_pl=eval_candidates)
print()

if not debug:
    eval_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'))

peak memory: 1626.96 MiB, increment: 0.10 MiB
Eval data

(64153741, 11)
Memory: 5.033693803474307
0.0031155782481960014
Validating

CPU times: user 56min 36s, sys: 3min 5s, total: 59min 42s
Wall time: 49min 1s


## test data

In [33]:
test_candidates = get_feature(target_df=test_cg_pl, data_type='test')

In [35]:
%load_ext memory_profiler

In [38]:
ranker_train_data_dir

'../data/rank_train_data_v2'

In [ ]:
%%time
%memit

test_candidates = test_candidates.collect()
print('Test data')
print()
print(test_candidates.shape)
print(f"Memory: {test_candidates.estimated_size(unit='gb')}")
print('Validating')
candidates_validation(target_pl=test_candidates)

if not debug:
    # eval_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'))
    test_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'))

peak memory: 1052.30 MiB, increment: 0.01 MiB


In [40]:
# test_candidates

In [ ]:
# eval_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'))
# test_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'))

### test4task3

In [51]:
! ls ../data/candidates/ | grep test4task3

task1_test4task3_co_visit_True_top300.parquet
task1_test4task3_next_few_items_v1_True_top100.parquet
task1_test4task3_next_item_counter_v2_True_top100.parquet
task1_test4task3_w2v_v3_True_top200.parquet
task2_test4task3_co_visit_task2_True_top300.parquet
task2_test4task3_nfi_task2_True_top100.parquet
task2_test4task3_nic_task2_True_top100.parquet
task2_test4task3_w2v_task2_True_top200.parquet


In [53]:
test4task3_cg_pl = get_all_candidates(data_type='test4task3')

Candidiate files: 
++++++++++++++++++++

../data/candidates/task1_test4task3_w2v_v3_True_top200.parquet
../data/candidates/task1_test4task3_nic_True_top100.parquet
../data/candidates/task1_test4task3_next_few_items_v1_True_top100.parquet
../data/candidates/task1_test4task3_co_visit_True_top300.parquet


In [54]:
# '../data/candidates/task1_test4task3_w2v_v3_True_top200.parquet'

In [56]:
# task1_test_w2v_v3_True_top100_phase1.parquet

In [57]:
test4task3_candidates = get_feature(target_df=test4task3_cg_pl, data_type='test4task3')
test4task3_candidates.collect().write_parquet(os.path.join(ranker_train_data_dir, 'test4task3.parquet'))

In [58]:
! ls ../data/candidates/ | grep test4task3

task1_test4task3_co_visit_True_top300.parquet
task1_test4task3_next_few_items_v1_True_top100.parquet
task1_test4task3_nic_True_top100.parquet
task1_test4task3_w2v_v3_True_top200.parquet
task2_test4task3_co_visit_task2_True_top300.parquet
task2_test4task3_nfi_task2_True_top100.parquet
task2_test4task3_nic_task2_True_top100.parquet
task2_test4task3_w2v_task2_True_top200.parquet


## Saving train data 

In [ ]:
# train_cg_pl.collect().sample(10)

In [ ]:
if debug:
    train_parts = 1
for part_index in tqdm(range(train_parts)):
    print()
    file_name = os.path.join(train_dir, f"part_{part_index+1}.parquet")
    print(file_name)
    train_cg_pl = get_all_candidates(data_type='train', part=part_index+1)
    sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', 
                                           negative_sampling=0.1
                                          )
    sampled_train_candidates = sampled_train_candidates.collect()
    print('Validating')
    candidates_validation(target_pl=sampled_train_candidates)
    # print('Sampled train data')
    print(sampled_train_candidates.shape)
    print(f"Memory: {sampled_train_candidates.estimated_size(unit='gb')}")
    print(sampled_train_candidates['target'].mean())
    
    stat_info = (
            sampled_train_candidates
                .groupby('session_id')
                .agg(
                    pl.count().alias('rec_cnt')
                    , pl.col('target').sum().alias('target_num')
                )
                .describe()
        )
    print(stat_info)
    if not debug:
        sampled_train_candidates.write_parquet(file_name)
    # del sampled_train_candidates

  0%|          | 0/20 [00:00<?, ?it/s]


../data/rank_train_data_v2/train/part_1.parquet
Original negative num: 28419506
Validating
(2923449, 11)
Memory: 0.22870463505387306
0.05013427632908937
shape: (7, 4)
┌────────────┬──────────────┬───────────┬────────────┐
│ describe   ┆ session_id   ┆ rec_cnt   ┆ target_num │
│ ---        ┆ ---          ┆ ---       ┆ ---        │
│ str        ┆ f64          ┆ f64       ┆ f64        │
╞════════════╪══════════════╪═══════════╪════════════╡
│ count      ┆ 146565.0     ┆ 146565.0  ┆ 146565.0   │
│ null_count ┆ 0.0          ┆ 0.0       ┆ 0.0        │
│ mean       ┆ 81754.567209 ┆ 19.946433 ┆ 1.0        │
│ std        ┆ 47232.319874 ┆ 9.651442  ┆ 0.0        │
│ min        ┆ 0.0          ┆ 8.0       ┆ 1.0        │
│ max        ┆ 163580.0     ┆ 47.0      ┆ 1.0        │
│ median     ┆ 81763.0      ┆ 16.0      ┆ 1.0        │
└────────────┴──────────────┴───────────┴────────────┘


  5%|▌         | 1/20 [06:13<1:58:17, 373.55s/it]


../data/rank_train_data_v2/train/part_2.parquet
Original negative num: 28439739
Validating
(2925578, 11)
Memory: 0.228871189057827
0.050086512819005335
shape: (7, 4)
┌────────────┬───────────────┬───────────┬────────────┐
│ describe   ┆ session_id    ┆ rec_cnt   ┆ target_num │
│ ---        ┆ ---           ┆ ---       ┆ ---        │
│ str        ┆ f64           ┆ f64       ┆ f64        │
╞════════════╪═══════════════╪═══════════╪════════════╡
│ count      ┆ 146532.0      ┆ 146532.0  ┆ 146532.0   │
│ null_count ┆ 0.0           ┆ 0.0       ┆ 0.0        │
│ mean       ┆ 245370.960937 ┆ 19.965455 ┆ 1.0        │
│ std        ┆ 47185.387385  ┆ 9.643766  ┆ 0.0        │
│ min        ┆ 163583.0      ┆ 8.0       ┆ 1.0        │
│ max        ┆ 327062.0      ┆ 47.0      ┆ 1.0        │
│ median     ┆ 245434.5      ┆ 16.0      ┆ 1.0        │
└────────────┴───────────────┴───────────┴────────────┘


 10%|█         | 2/20 [12:39<1:54:19, 381.08s/it]

...


../data/rank_train_data_v2/train/part_4.parquet
Original negative num: 28418720
Validating
(2923489, 11)
Memory: 0.22870776429772377
0.05014624648835689
shape: (7, 4)
┌────────────┬───────────────┬───────────┬────────────┐
│ describe   ┆ session_id    ┆ rec_cnt   ┆ target_num │
│ ---        ┆ ---           ┆ ---       ┆ ---        │
│ str        ┆ f64           ┆ f64       ┆ f64        │
╞════════════╪═══════════════╪═══════════╪════════════╡
│ count      ┆ 146602.0      ┆ 146602.0  ┆ 146602.0   │
│ null_count ┆ 0.0           ┆ 0.0       ┆ 0.0        │
│ mean       ┆ 572286.914333 ┆ 19.941672 ┆ 1.0        │
│ std        ┆ 47163.728377  ┆ 9.643425  ┆ 0.0        │
│ min        ┆ 490554.0      ┆ 9.0       ┆ 1.0        │
│ max        ┆ 654092.0      ┆ 47.0      ┆ 1.0        │
│ median     ┆ 572259.5      ┆ 16.0      ┆ 1.0        │
└────────────┴───────────────┴───────────┴────────────┘


 20%|██        | 4/20 [25:34<1:42:45, 385.31s/it]


../data/rank_train_data_v2/train/part_5.parquet
Original negative num: 28108705
Validating
(2892439, 11)
Memory: 0.22627868875861168
0.050644110385733285
shape: (7, 4)
┌────────────┬───────────────┬───────────┬────────────┐
│ describe   ┆ session_id    ┆ rec_cnt   ┆ target_num │
│ ---        ┆ ---           ┆ ---       ┆ ---        │
│ str        ┆ f64           ┆ f64       ┆ f64        │
╞════════════╪═══════════════╪═══════════╪════════════╡
│ count      ┆ 146485.0      ┆ 146485.0  ┆ 146485.0   │
│ null_count ┆ 0.0           ┆ 0.0       ┆ 0.0        │
│ mean       ┆ 735873.186292 ┆ 19.745633 ┆ 1.0        │
│ std        ┆ 47216.481088  ┆ 9.522173  ┆ 0.0        │
│ min        ┆ 654093.0      ┆ 9.0       ┆ 1.0        │
│ max        ┆ 817747.0      ┆ 47.0      ┆ 1.0        │
│ median     ┆ 735805.0      ┆ 16.0      ┆ 1.0        │
└────────────┴───────────────┴───────────┴────────────┘


 25%|██▌       | 5/20 [32:13<1:37:32, 390.19s/it]


../data/rank_train_data_v2/train/part_6.parquet


# Slow speed -> **7 hours to finished all of the data**



```
../data/rank_train_data_v2/train/part_1.parquet
Validating
(28566071, 11)
Memory: 2.23475506529212
0.005130737090165463
shape: (7, 4)
┌────────────┬──────────────┬───────────┬────────────┐
│ describe   ┆ session_id   ┆ rec_cnt   ┆ target_num │
│ ---        ┆ ---          ┆ ---       ┆ ---        │
│ str        ┆ f64          ┆ f64       ┆ f64        │
╞════════════╪══════════════╪═══════════╪════════════╡
│ count      ┆ 146565.0     ┆ 146565.0  ┆ 146565.0   │
│ null_count ┆ 0.0          ┆ 0.0       ┆ 0.0        │
│ mean       ┆ 81754.567209 ┆ 194.90377 ┆ 1.0        │
│ std        ┆ 47232.319874 ┆ 96.523984 ┆ 0.0        │
│ min        ┆ 0.0          ┆ 80.0      ┆ 1.0        │
│ max        ┆ 163580.0     ┆ 468.0     ┆ 1.0        │
│ median     ┆ 81763.0      ┆ 154.0     ┆ 1.0        │
└────────────┴──────────────┴───────────┴────────────┘
  5%|▌         | 1/20 [22:21<7:04:44, 1341.26s/it]

../data/rank_train_data_v2/train/part_2.parquet

```

# Eval

In [ ]:
# ! ls {ranker_train_data_dir}

In [ ]:
# ! rm {ranker_train_data_dir}/*

In [ ]:
# pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test.parquet')).collect().shape

In [ ]:
# pl.scan_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet')).collect().shape

# Debug 

In [43]:
# part_index = 0
# print()
# file_name = os.path.join(train_dir, f"part_{part_index+1}.parquet")
# print(file_name)
# train_cg_pl = get_all_candidates(data_type='train', part=part_index+1)
# sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', 
#                                        # negative_sampling=0.1
#                                       )
# sampled_train_candidates = sampled_train_candidates.collect()
# print('Validating')
# candidates_validation(target_pl=sampled_train_candidates)
# # print('Sampled train data')
# print(sampled_train_candidates.shape)
# print(f"Memory: {sampled_train_candidates.estimated_size(unit='gb')}")
# print(sampled_train_candidates['target'].mean())

# stat_info = (
#         sampled_train_candidates
#             .groupby('session_id')
#             .agg(
#                 pl.count().alias('rec_cnt')
#                 , pl.col('target').sum().alias('target_num')
#             )
#             .describe()
#     )
# print(stat_info)
# (
#         train_cg_pl.select(
#                 pl.col('next_item_prediction').arr.lengths().alias('rec_num')
#             )
#             .collect()
#             .describe(
#                 # percentiles=[0.1*i for i in range(10)]
#             )
#     )


../data/rank_train_data_v2/train/part_1.parquet
Validating
(335696, 11)
Memory: 0.02626188099384308
0.010298007721271627
shape: (7, 4)
┌────────────┬─────────────┬───────────┬────────────┐
│ describe   ┆ session_id  ┆ rec_cnt   ┆ target_num │
│ ---        ┆ ---         ┆ ---       ┆ ---        │
│ str        ┆ f64         ┆ f64       ┆ f64        │
╞════════════╪═════════════╪═══════════╪════════════╡
│ count      ┆ 3457.0      ┆ 3457.0    ┆ 3457.0     │
│ null_count ┆ 0.0         ┆ 0.0       ┆ 0.0        │
│ mean       ┆ 5540.907723 ┆ 97.106161 ┆ 1.0        │
│ std        ┆ 3186.084851 ┆ 1.80037   ┆ 0.0        │
│ min        ┆ 5.0         ┆ 79.0      ┆ 1.0        │
│ max        ┆ 11052.0     ┆ 100.0     ┆ 1.0        │
│ median     ┆ 5470.0      ┆ 98.0      ┆ 1.0        │
└────────────┴─────────────┴───────────┴────────────┘


describe,rec_num
str,f64
"""count""",10000.0
"""null_count""",0.0
"""mean""",97.7283
"""std""",1.868804
"""min""",79.0
"""max""",100.0
"""median""",98.0


In [45]:
# (train_cg_pl.with_columns(
#         pl.col('next_item_prediction').arr.lengths().alias('rec_num')
#     )
#     .filter(pl.col('rec_num') == 100)
#     .head().collect())

session_id,next_item_prediction,w2v_weight,next_item,locale,prev_length,last_prev_item,rec_num
i64,list[str],list[f64],str,str,u32,str,u32
9184,"[""B08447LBF5"", ""B073VPDRP1"", … ""B08FB3KWXL""]","[0.868687, 0.515152, … 0.828283]","""B017RJW8OW""","""DE""",3,"""B09XF26MW6""",100
5152,"[""B01LZPR94B"", ""B0B6H7NDBS"", … ""B0B9X73PB5""]","[0.787879, 0.515152, … 0.79798]","""B0B8S79JGZ""","""DE""",3,"""B0BJPZHZRB""",100
5920,"[""B09MHXJ4QK"", ""B00C81E9OW"", … ""B0B372LV18""]","[0.393939, 0.585859, … 0.262626]","""B098DY9JS2""","""DE""",2,"""B094TZ4V98""",100
8480,"[""B01E5A7KIK"", ""B094JG3PBD"", … ""B07Z9H23XK""]","[0.181818, 0.373737, … 0.515152]","""B014PI4HGK""","""DE""",2,"""B0B5GZYMJH""",100
1152,"[""B095R6SRG7"", ""B07NY59JGM"", … ""B08BXH5P73""]","[0.353535, 0.464646, … 0.262626]","""B07WPY21TW""","""DE""",2,"""B00FYV34FI""",100


In [65]:
# search_id = 9184

In [48]:
# data_type = 'train'
# part = 0 +1 
# w2v_file = os.path.join(candidate_path, 
#                        candidate_file_name.format(
#                 task=task
#             , data_type=data_type
#             , model_version=w2v_model_version
#             , model_for_eval=model_for_eval
#             , topn=w2v_topn
#                        ))
# nic_file = os.path.join(candidate_path,
#             candidate_file_name.format(
#                 task=task
#                 , data_type=data_type
#                 , model_version=nic_model_version
#                 , model_for_eval=model_for_eval
#                 , topn=nic_topn
#                        ))
# nfi_file = os.path.join(candidate_path,
#         candidate_file_name.format(
#             task=task
#             , data_type=data_type
#             , model_version=nfi_model_version
#             , model_for_eval=model_for_eval
#             , topn=nfi_topn
#                    ))
# covisit_file = os.path.join(candidate_path,
#         candidate_file_name.format(
#             task=task
#             , data_type=data_type
#             , model_version=covisit_model_version
#             , model_for_eval=model_for_eval
#             , topn=covisit_topn
#                    ))
# if data_type == 'train':
#     assert part is not None
#     w2v_file = os.path.join(w2v_file.replace('.parquet', ''), f"part_{part}.parquet")
#     nic_file = os.path.join(nic_file.replace('.parquet', ''), f"part_{part}.parquet")
#     nfi_file = os.path.join(nfi_file.replace('.parquet', ''), f"part_{part}.parquet")
#     covisit_file = os.path.join(covisit_file.replace('.parquet', ''), f"part_{part}.parquet")

# if data_type == 'test':
#     original_file_name = f"sessions_{data_type}_{task}.parquet"
# else:
#     original_file_name = f"sessions_{data_type}.parquet"
# original_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, original_file_name), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
# w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
# nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
# nfi_pl = pl.scan_parquet(nfi_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
# covisit_pl = pl.scan_parquet(covisit_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))


# #     def get_vector(prev_items):
# #         prev_items_vec = np.array([w2vec.wv[ele] for ele in prev_items]).mean(axis=0)
# #         return prev_items_vec

# #     original_pl = (
# #             original_pl.with_columns(
# #                 pl.col('prev_items').apply(lambda x: get_vector(x)).alias('all_previous_items_vec')
# #             )
# #         )

# # get w2v weight
# w2v_pl = w2v_pl.with_columns(pl.col('next_item_prediction').arr.lengths().alias('rec_num')).with_columns(
#     pl.col('rec_num').apply(lambda x: list(range(x, 0, -1))).alias('w2v_weight')

# )
# # print(w2v_pl.head().collect())
# get_w2v_weight = pl.element().rank()*0
# nic_pl = nic_pl.with_columns(
#     pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
#                                             # parallel=True
#                                            ).alias('w2v_weight').cast(pl.List(pl.Int64))
# )
# nfi_pl = nfi_pl.with_columns(
#     pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
#                                            ).alias('w2v_weight').cast(pl.List(pl.Int64))
# )
# covisit_pl = covisit_pl.with_columns(
#     pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
#                                            ).alias('w2v_weight').cast(pl.List(pl.Int64))
# )
# cols = ['session_id', 'next_item_prediction', 
#         'w2v_weight'
#        ]


In [57]:
# nic_rec = nic_pl.filter(pl.col('session_id')==search_id).select('next_item_prediction').collect().to_series().to_list()[0]
# w2v_rec = w2v_pl.filter(pl.col('session_id')==search_id).select('next_item_prediction').collect().to_series().to_list()[0]